In [1]:
# Necessary libraries

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

px.defaults.template = "plotly_white"
print("Libraries loaded.")


Libraries loaded.


In [2]:
# Load dataset 

df = pd.read_csv("Energy dataset.csv")
print("Raw rows:", len(df))
df.head()


Raw rows: 43848


,Timestamp,Household_ID,Room,Appliance,Usage_KWh
0,2025-03-01 00:00:00,H001,Living Room,AC,0.448
1,2025-03-01 00:00:00,H001,Kitchen,Refrigerator,0.207
2,2025-03-01 00:00:00,H001,Living Room,TV,0.049
3,2025-03-01 00:00:00,H001,Bedroom,Lights,0.243
4,2025-03-01 00:00:00,H001,Bedroom,Fan,0.194


In [3]:
# Preprocessing

# a) Timestamp parse
ts_col_candidates = [c for c in df.columns if "time" in c.lower() or "date" in c.lower() or "timestamp" in c.lower()]
if not ts_col_candidates:
    raise ValueError("No timestamp-like column found.")
ts_col = ts_col_candidates[0]
df = df.rename(columns={ts_col: "Timestamp"})
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# b) Ensure expected columns
expected = ["Household_ID", "Room", "Appliance", "Usage_KWh"]
for col in expected:
    if col not in df.columns:
        print(f"Warning: {col} not found in CSV.")

# c) Force numeric usage and sort
df["Usage_KWh"] = pd.to_numeric(df["Usage_KWh"], errors="coerce").fillna(0.0)
df = df.sort_values("Timestamp").reset_index(drop=True)

# d) Create hourly & daily aggregates for analysis / forecasting
hourly = df.set_index("Timestamp").resample("H")["Usage_KWh"].sum().reset_index()
daily  = df.set_index("Timestamp").resample("D")["Usage_KWh"].sum().reset_index()

print("After preprocess — hourly rows:", len(hourly), "daily rows:", len(daily))
hourly.head(), daily.head()


After preprocess — hourly rows: 6264 daily rows: 261


(            Timestamp  Usage_KWh
 0 2025-03-01 00:00:00      1.141
 1 2025-03-01 01:00:00      1.113
 2 2025-03-01 02:00:00      0.846
 3 2025-03-01 03:00:00      1.090
 4 2025-03-01 04:00:00      1.309,
    Timestamp  Usage_KWh
 0 2025-03-01     45.822
 1 2025-03-02     43.488
 2 2025-03-03     42.549
 3 2025-03-04     42.570
 4 2025-03-05     45.136)

In [4]:
#  Analysis 

# Appliance-wise total (bar)
appliance_tot = df.groupby("Appliance")["Usage_KWh"].sum().reset_index().sort_values("Usage_KWh", ascending=False)
figA = px.bar(appliance_tot, x="Appliance", y="Usage_KWh", title="Total consumption by appliance (kWh)")
figA.show()





In [5]:
# Room-wise share (pie)
room_tot = df.groupby("Room")["Usage_KWh"].sum().reset_index().sort_values("Usage_KWh", ascending=False)
figB = px.pie(room_tot, names="Room", values="Usage_KWh", title="Energy share by room")
figB.show()

In [6]:
# Hourly time series (last 30 days) + Heatmap
# Time series
figC = px.line(hourly.tail(24*30), x="Timestamp", y="Usage_KWh", title="Hourly usage — last 30 days")
figC.show()

In [7]:
# Heatmap: Hour vs Day of week (average)
h = hourly.copy()
h["hour"] = h["Timestamp"].dt.hour
h["dow"] = h["Timestamp"].dt.day_name()
pivot = h.pivot_table(index="hour", columns="dow", values="Usage_KWh", aggfunc="mean")
# reorder columns Mon..Sun if present
order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
cols_present = [c for c in order if c in pivot.columns]
pivot = pivot[cols_present]
figH = go.Figure(data=go.Heatmap(z=pivot.values, x=pivot.columns, y=pivot.index, colorbar=dict(title="kWh (avg)")))
figH.update_layout(title="Avg hourly usage — Hour vs Day of week", xaxis_title="Day", yaxis_title="Hour")
figH.show()

In [8]:
# Train/test prep — daily
df_daily = daily.copy().rename(columns={"Usage_KWh":"Kwh"})
df_daily["day"] = df_daily["Timestamp"].dt.day
df_daily["month"] = df_daily["Timestamp"].dt.month
df_daily["year"] = df_daily["Timestamp"].dt.year
df_daily["dayofweek"] = df_daily["Timestamp"].dt.dayofweek
df_daily["is_weekend"] = df_daily["dayofweek"].isin([5,6]).astype(int)

# Lag & rolling features
df_daily["lag_1"] = df_daily["Kwh"].shift(1)
df_daily["lag_7"] = df_daily["Kwh"].shift(7)
df_daily["lag_30"] = df_daily["Kwh"].shift(30)
df_daily["roll_7"] = df_daily["Kwh"].rolling(7).mean()
df_daily["roll_30"] = df_daily["Kwh"].rolling(30).mean()

# Drop NA rows created by lags
df_daily = df_daily.dropna().reset_index(drop=True)
print("Rows after feature creation:", len(df_daily))
df_daily.tail()


Rows after feature creation: 231


,Timestamp,Kwh,day,month,year,dayofweek,is_weekend,lag_1,lag_7,lag_30,roll_7,roll_30
226,2025-11-12,40.863,12,11,2025,2,0,43.345,49.856,43.994,44.148714,44.533533
227,2025-11-13,44.403,13,11,2025,3,0,40.863,48.417,44.562,43.575286,44.528233
228,2025-11-14,46.126,14,11,2025,4,0,44.403,39.939,38.498,44.459143,44.782500
229,2025-11-15,45.519,15,11,2025,5,1,46.126,42.639,44.524,44.870571,44.815667
230,2025-11-16,47.596,16,11,2025,6,1,45.519,50.725,45.750,44.423571,44.877200


In [9]:
# Build Linear Regression model (train/test split)
features = ["day","month","year","dayofweek","is_weekend","lag_1","lag_7","lag_30","roll_7","roll_30"]
X = df_daily[features]
y = df_daily["Kwh"]

# Time-based split: use first 85% for train, last 15% for test
split_idx = int(len(X) * 0.85)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate
y_pred = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Train rows: {len(X_train)}, Test rows: {len(X_test)}")
print(f"LinearRegression MAE: {mae:.3f} kWh, RMSE: {rmse:.3f} kWh")


Train rows: 196, Test rows: 35
LinearRegression MAE: 2.327 kWh, RMSE: 2.789 kWh


In [10]:
# Smart tips (simple rule-based)
def generate_smart_tips(df_full, appliance_tot, room_tot, hourly_df=None):
    tips = []

    # Always use correct column name
    total = df_full["Usage_kWh"].sum()

    # -------------------------------
    # Top 3 appliances
    # -------------------------------
    top_apps = appliance_tot.sort_values("Usage_kWh", ascending=False).head(3)

    for _, row in top_apps.iterrows():
        app = row["Appliance"]
        pct = row["Usage_kWh"] / total * 100 if total > 0 else 0
        tips.append(
            f"{app} uses {pct:.1f}% of total energy — consider efficient settings or scheduling."
        )

    # -------------------------------
    # Peak hour detection
    # -------------------------------
    if hourly_df is not None:
        hourly_df["hour"] = hourly_df["Timestamp"].dt.hour
        peak_hour = int(
            hourly_df.groupby("hour")["Usage_kWh"].mean().idxmax()
        )
        tips.append(f"Peak hour around {peak_hour}:00 — shift discretionary loads (washer/microwave) away from this hour.")

    # -------------------------------
    # Refrigerator tip
    # -------------------------------
    if "Refrigerator" in appliance_tot["Appliance"].values:
        fridge_val = appliance_tot.loc[
            appliance_tot["Appliance"] == "Refrigerator", "Usage_kWh"
        ].values[0]

        if total > 0 and fridge_val / total * 100 > 20:
            tips.append(
                "Refrigerator >20% of usage — check door seals, clean coils, or raise thermostat by 1–2°C."
            )

    # -------------------------------
    # Lights tip (FIXED!)
    # -------------------------------
    if "Lights" in appliance_tot["Appliance"].values:
        lights_val = appliance_tot.loc[
            appliance_tot["Appliance"] == "Lights", "Usage_kWh"
        ].values[0]

        if total > 0 and lights_val / total * 100 > 5:
            tips.append(
                "Lights seem high — consider LED bulbs, daylight use, or motion sensors."
            )

    return tips




In [11]:
# Forecast next 2/3 months (daily) using Linear Regression and append predictions column to a new dataframe.
months_to_forecast = 2   # change to 3 for 3 months
days_to_forecast = months_to_forecast * 30  # approx (you can use calendar month logic if needed)

# Prepare temp dataframe to iteratively forecast
temp = df_daily.copy().reset_index(drop=True)
future_rows = []

last_date = temp["Timestamp"].max()

for i in range(days_to_forecast):
    next_date = last_date + pd.Timedelta(days=1)

    # Build feature row
    feat_row = {
        "Timestamp": next_date,
        "day": next_date.day,
        "month": next_date.month,
        "year": next_date.year,
        "dayofweek": next_date.dayofweek,
        "is_weekend": int(next_date.dayofweek in [5,6])
    }

    # Lags from temp
    feat_row["lag_1"]  = temp.iloc[-1]["Kwh"]
    feat_row["lag_7"]  = temp.iloc[-7]["Kwh"] if len(temp) >= 7 else feat_row["lag_1"]
    feat_row["lag_30"] = temp.iloc[-30]["Kwh"] if len(temp) >= 30 else feat_row["lag_1"]

    # Rolling averages
    feat_row["roll_7"]  = temp["Kwh"].tail(7).mean()
    feat_row["roll_30"] = temp["Kwh"].tail(30).mean()

    # Prediction
    X_new = pd.DataFrame([feat_row])[features]
    pred = lr.predict(X_new)[0]
    feat_row["Forecast_Kwh"] = float(np.round(pred, 3))

    # -------- FIXED: Use concat instead of append --------
    new_row = pd.DataFrame([{
        "Timestamp": next_date,
        "Kwh": feat_row["Forecast_Kwh"],
        "day": feat_row["day"],
        "month": feat_row["month"],
        "year": feat_row["year"],
        "dayofweek": feat_row["dayofweek"],
        "is_weekend": feat_row["is_weekend"]
    }])

    temp = pd.concat([temp, new_row], ignore_index=True)
    # ------------------------------------------------------

    # Store forecast row
    future_rows.append(feat_row)
    last_date = next_date

# Convert list of dicts to DataFrame
future_df = pd.DataFrame(future_rows)

print(f"Forecast rows: {len(future_df)} — from {future_df['Timestamp'].min().date()} to {future_df['Timestamp'].max().date()}")



Forecast rows: 60 — from 2025-11-17 to 2026-01-15


In [12]:
# Combine actual last 30 days + forecast
plot_actual   = df_daily[["Timestamp", "Kwh"]].tail(30).rename(columns={"Kwh": "Value"})
plot_forecast = future_df[["Timestamp", "Forecast_Kwh"]].rename(columns={"Forecast_Kwh": "Value"})

plot_df = pd.concat([plot_actual, plot_forecast], ignore_index=True)

fig = px.line(
    plot_df, x="Timestamp", y="Value",
    title=f"Last 30 days actual + Next {months_to_forecast} months forecast (daily)",
    labels={"Value": "kWh"}
)
fig.show()


In [13]:
# Create combined aligned dataframe
historic = df_daily[["Timestamp", "Kwh"]].rename(columns={"Kwh": "Actual_kwh"})
forecasted = future_df[["Timestamp", "Forecast_Kwh"]].rename(columns={"Forecast_Kwh": "Predicted_kwh"})

daily_all = pd.merge(
    historic,
    forecasted,
    on="Timestamp",
    how="outer"
).sort_values("Timestamp").reset_index(drop=True)

daily_all.tail(10)


,Timestamp,Actual_kwh,Predicted_kwh
281,2026-01-06,NaN,44.508
282,2026-01-07,NaN,44.556
283,2026-01-08,NaN,44.620
284,2026-01-09,NaN,44.540
285,2026-01-10,NaN,44.353
286,2026-01-11,NaN,44.313
287,2026-01-12,NaN,44.896
288,2026-01-13,NaN,44.833
289,2026-01-14,NaN,44.848
290,2026-01-15,NaN,44.824
